In [23]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [36]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

# Upload data for training

In [32]:
!git clone https://github.com/facebookresearch/detectron2

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [34]:
! ./detectron2/datasets/prepare_for_tests.sh

coco/annotations/instances_val2017_100.json exists. Skipping ...
coco/annotations/person_keypoints_val2017_100.json exists. Skipping ...
coco/annotations/instances_minival2014_100.json exists. Skipping ...
coco/annotations/person_keypoints_minival2014_100.json exists. Skipping ...


In [58]:
# upload directory with COCO dataset to S3 location

import os
import boto3

s3_resource = boto3.resource("s3", region_name="us-east-2")
root_path = os.getcwd()
data_path = os.path.join(root_path, "detectron2/datasets/coco")



for path, subdirs, files in os.walk(data_path):

    directory_name = path.replace(root_path+"/","")
    for file in files:
        print(os.path.join(root_path, directory_name, file))
        s3_resource.Bucket(bucket).upload_file(os.path.join(root_path, directory_name, file), directory_name+'/'+file)
        print(f"file {file} has been uploaded to S3")

/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/person_keypoints_val2017_100.json
file person_keypoints_val2017_100.json has been uploaded to S3
/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/instances_minival2014_100.json
file instances_minival2014_100.json has been uploaded to S3
/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/person_keypoints_minival2014_100.json
file person_keypoints_minival2014_100.json has been uploaded to S3
/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/instances_val2017_100.json
file instances_val2017_100.json has been uploaded to S3


# Train your model

In [62]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = "us-east-2"
image = '{}.dkr.ecr.{}.amazonaws.com/detectron2-sagemaker:latest'.format(account, region)

d2 = sagemaker.estimator.Estimator(image,
                                   role=role,
                                   train_instance_count=1, 
                                   train_instance_type='ml.p3.2xlarge',
                                   output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                   sagemaker_session=sess)

d2.set_hyperparameters(num_epochs = 1, num_classes = 2, )

d2.fit(f"s3://{bucket}/detectron2/dataset") 

2020-03-31 17:26:34 Starting - Starting the training job...
2020-03-31 17:26:35 Starting - Launching requested ML instances...
2020-03-31 17:27:29 Starting - Preparing the instances for training.........
2020-03-31 17:29:00 Downloading - Downloading input data...
2020-03-31 17:29:11 Training - Downloading the training image..............
2020-03-31 17:32:05 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-31 17:31:51,491 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-31 17:31:51,515 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-03-31 17:31:51,515 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-03-31 17:31:52,010 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-03-31 17:31:52,010

UnexpectedStatusException: Error for Training job detectron2-sagemaker-2020-03-31-17-26-34-419: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/bin/sh -c ./train --num_classes 2 --num_epochs 1"
Traceback (most recent call last):
  File "./train_net.py", line 163, in <module>
    args=(args,),
  File "/opt/ml/code/detectron2/detectron2/engine/launch.py", line 52, in launch
    main_func(*args)
  File "./train_net.py", line 145, in main
    trainer = Trainer(cfg)
  File "/opt/ml/code/detectron2/detectron2/engine/defaults.py", line 255, in __init__
    data_loader = self.build_train_loader(cfg)
  File "/opt/ml/code/detectron2/detectron2/engine/defaults.py", line 430, in build_train_loader
    return build_detection_train_loader(cfg)
  File "/opt/ml/code/detectron2/detectron2/data/build.py", line 297, in build_detection_train_loader
    proposal_files=cfg.DATASETS.PROPOSAL_FILES_TRAIN if cfg.MODEL.LOAD_PROPOSALS else None,
  File "/opt/ml/code/detectron2/detectron2/data/b

In [61]:
#TODO fix the issue with hostnames: https://github.com/aws/sagemaker-pytorch-container/issues/143
